## 1 -Import models

In [1]:
import pandas_datareader as data_reader
import math
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook,tqdm
from collections import deque
tf.__version__


'2.5.0'

## 2-Build AI class trader(functions+ANN)

In [22]:
class AI_trader():
    def __init__(self,state_size,action_space=3,model_name="AItrader"):
        self.state_size=state_size
        self.action_space=action_space
        self.memory=deque(maxlen=2000)
        self.inventory=[]
        self.model_name=model_name
        
        self.gamma=0.95
        self.epsilon=1.0
        self.epsilon_final=0.01
        self.epsilon_decay=0.995
        
        self.model=self.model_builder()
        
        # function model builder
    def model_builder(self):
        model=tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(units=32,activation="relu",input_dim=self.state_size))
        model.add(tf.keras.layers.Dense(units=64,activation="relu"))
        model.add(tf.keras.layers.Dense(units=128,activation="relu"))
        model.add(tf.keras.layers.Dense(units=self.action_space,activation="linear"))
        model.compile(loss="mse",optimizer=tf.keras.optimizers.Adam(lr=0.001))
        return model
    
        # function trade
    def trade(self,state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)
        
        actions=self.model.predict(state)
        return np.argmax(actions[0])
    
        #function batch train
    def batch_train(self,batch_size):
        batch=[]
        for i in range(len(self.memory)-batch_size+1,len(self.memory)):
            batch.append(self.memory[i])

        for state,action,reward,next_state,done in batch:
            reward=reward
            if not done:
                reward=reward+self.gamma*np.amax(self.model.predict(next_state)[0])
            target=self.model.predict(state)
            target[0][action]=reward
            self.model.fit(state,target,epochs=1,verbose=1)
        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay

## 3-define dataset loader function

In [24]:
def dataset_loader(stock_name):
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset['Close']
  return close    

In [26]:
stock_name="AAPL"
data=dataset_loader(stock_name)

In [27]:
data

Date
2016-06-20     23.775000
2016-06-21     23.977501
2016-06-22     23.887501
2016-06-23     24.025000
2016-06-24     23.350000
                 ...    
2021-06-11    127.349998
2021-06-14    130.479996
2021-06-15    129.639999
2021-06-16    130.149994
2021-06-17    132.029999
Name: Close, Length: 1258, dtype: float64

## 4- define state creator function

In [28]:
def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])


## 5- define sigmoid function¶

In [30]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

## 6- define stock price function

In [31]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

## 7-Setting hyperparameter

In [32]:
window_size=10
episodes=1000
batch_size=32
data_samples =len(data)-1

## 8-defining the trader model

In [33]:
trader=AI_trader(window_size)
trader.model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                352       
_________________________________________________________________
dense_5 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


C:\Users\abdou\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


## 8-Train the model

In [34]:
for episode in range(1,episodes+1):
    print("Episodes: {}/{}".format(episode,episodes))
    state=state_creator(data,0,window_size+1)
    total_profit=0
    trader.inventory=[]
    for t in tqdm(range(data_samples)):
        action=trader.trade(state)
        next_state=state_creator(data,t+1,window_size+1)
        reward=0
        
        if action==1:
            trader.inventory.append(data[t])
            print("AI trader bought:",stocks_price_format(data[t]))
        elif action==2 and len(trader.inventory)>0:
            buy_price=trader.inventory.pop(0)
            print("AI trader sold:",stocks_price_format(data[t]),"Profit"+stocks_price_format(data[t]-buy_price))
            reward=max(data[t]-buy_price,0)
            total_profit += data[t]-buy_price
            
        if t==data_samples-1:
            done=True
        else:
            done=False
        trader.memory.append((state,action,reward,next_state,done))
        state=next_state
        if done:
            print("####################################")
            print("TOTAL PROFIT:{}".format(total_profit))
            print("####################################")
        if len(trader.memory)>batch_size:
            trader.batch_train(batch_size)
    if episode % 10 == 0:
        trader.model.save("ai_trader_{}.h5".format(episode))

  0%|          | 0/1257 [00:00<?, ?it/s]

Episodes: 1/1000
AI trader bought: $ 24.025000
AI trader sold: $ 23.350000 Profit- $ 0.674999
AI trader bought: $ 23.900000
AI trader sold: $ 23.972500 Profit$ 0.072500
AI trader bought: $ 23.747499
AI trader sold: $ 23.882500 Profit$ 0.135000
AI trader bought: $ 24.355000
AI trader bought: $ 24.697500
AI trader sold: $ 24.957500 Profit$ 0.602501
AI trader sold: $ 24.967501 Profit$ 0.270000
AI trader bought: $ 24.665001
AI trader bought: $ 25.737499
AI trader sold: $ 26.084999 Profit$ 1.419998
AI trader bought: $ 26.052500
AI trader sold: $ 26.447500 Profit$ 0.710001
AI trader sold: $ 26.467501 Profit$ 0.415001
1/1 [==============================] - 0s 6ms/step - loss: 0.0473


  3%|▎         | 33/1257 [00:05<03:38,  5.61it/s]

1/1 [==============================] - 0s 5ms/step - loss: 0.0173


  3%|▎         | 34/1257 [00:09<06:50,  2.98it/s]

...

AI trader sold: $ 29.264999 Profit- $ 0.070000
1/1 [==============================] - 0s 5ms/step - loss: 0.2022


  7%|▋         | 87/1257 [03:30<1:11:29,  3.67s/it]

AI trader sold: $ 29.150000 Profit- $ 0.217501
1/1 [==============================] - 0s 5ms/step - loss: 0.0567


  7%|▋         | 87/1257 [03:30<47:09,  2.42s/it]  


KeyboardInterrupt: 